# Player Performance Prediction (QB) - Transformer + Time2Vec

This notebook implements a Transformer-based model with Time2Vec embeddings to predict future player performance (e.g., PFF Grade) based on historical data. 

## Architecture
1. **Time2Vec Embedding**: Captures periodic and linear temporal patterns.
2. **Transformer Encoder**: Captures long-range dependencies and interactions between features.
3. **Regression Head**: Predicts the target metric.

In [20]:
# Install dependencies if not already installed
!pip install tensorflow pandas numpy scikit-learn matplotlib

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import os
import matplotlib.pyplot as plt

## 1. Time2Vec Layer

In [22]:
class Time2Vec(layers.Layer):
    def __init__(self, kernel_size=1, **kwargs):
        super(Time2Vec, self).__init__(**kwargs)
        self.k = kernel_size
    
    def build(self, input_shape):
        # weights: (k+1) inputs (1 for linear, k for periodic)
        self.wb = self.add_weight(name='wb',
                                shape=(input_shape[-1],),
                                initializer='uniform',
                                trainable=True)
        self.wa = self.add_weight(name='wa',
                                shape=(1, input_shape[-1], self.k),
                                initializer='uniform',
                                trainable=True)
        self.ba = self.add_weight(name='ba',
                                shape=(1, input_shape[-1], self.k),
                                initializer='uniform',
                                trainable=True)
        super(Time2Vec, self).build(input_shape)

    def call(self, inputs, **kwargs):
        # inputs shape: (batch_size, time_steps, features)
        # Linear term: wb * inputs
        bias = self.wb * inputs
        # Periodic term: sin(wa * inputs + ba)
        pattern = tf.math.sin(tf.matmul(inputs, self.wa) + self.ba)
        pattern = tf.reshape(pattern, (-1, inputs.shape[1], inputs.shape[2] * self.k))
        return tf.concat([bias, pattern], axis=-1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], input_shape[2] * (self.k + 1))

## 2. Transformer Block

In [23]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = models.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

## 3. Data Loading & Sliding Window Generation

We need to group by player and create sequences (e.g., Year 1-3 predicts Year 4).

In [24]:
# Load Data from the specified path
data_path = '../../Grouped_Data/Grouped_QB.csv' 
if not os.path.exists(data_path):
    # Attempt absolute path fallback
    data_path = '/Users/pranaynandkeolyar/Documents/NFLSalaryCap/backend/Grouped_Data/Grouped_QB.csv'

print(f"Reading data from: {data_path}")
df = pd.read_csv(data_path)
print(f"Loaded Data Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")  # Print columns to debug KeyError

# Ensure 'Year' column exists and handle potential variations
if 'year' in df.columns: 
    df.rename(columns={'year': 'Year'}, inplace=True)

print(f"Years covered: {df['Year'].min()} - {df['Year'].max()}")

# Check Player Identifier
# The file might use 'Name', 'Player', or 'player'. 
possible_id_cols = ['player', 'Player', 'Name', 'name']
player_id_col = None
for col in possible_id_cols:
    if col in df.columns:
        player_id_col = col
        break

if player_id_col:
    print(f"Using player identifier column: {player_id_col}")
    df.rename(columns={player_id_col: 'player'}, inplace=True)
else:
    raise KeyError(f"Could not find a player identifier column. Available columns: {df.columns.tolist()}")

print(f"Unique Players: {df['player'].nunique()}")
df.sort_values(by=['player', 'Year'], inplace=True)
df.head()

Reading data from: ../../Grouped_Data/Grouped_QB.csv
Loaded Data Shape: (480, 43)
Columns: ['Unnamed: 0', 'Team', 'Year', 'Cap_Space', 'adjusted_value', 'Net EPA', 'Win %', 'franchise_id', 'aimed_passes', 'attempts', 'completions', 'touchdowns', 'interceptions', 'sacks', 'scrambles', 'first_downs', 'yards', 'dropbacks', 'drops', 'big_time_throws', 'turnover_worthy_plays', 'bats', 'declined_penalties', 'penalties', 'hit_as_threw', 'thrown_aways', 'spikes', 'accuracy_percent', 'completion_percent', 'btt_rate', 'twp_rate', 'drop_rate', 'qb_rating', 'sack_percent', 'pressure_to_sack_rate', 'avg_depth_of_target', 'avg_time_to_throw', 'ypa', 'def_gen_pressures', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run']
Years covered: 2010 - 2024


KeyError: "Could not find a player identifier column. Available columns: ['Unnamed: 0', 'Team', 'Year', 'Cap_Space', 'adjusted_value', 'Net EPA', 'Win %', 'franchise_id', 'aimed_passes', 'attempts', 'completions', 'touchdowns', 'interceptions', 'sacks', 'scrambles', 'first_downs', 'yards', 'dropbacks', 'drops', 'big_time_throws', 'turnover_worthy_plays', 'bats', 'declined_penalties', 'penalties', 'hit_as_threw', 'thrown_aways', 'spikes', 'accuracy_percent', 'completion_percent', 'btt_rate', 'twp_rate', 'drop_rate', 'qb_rating', 'sack_percent', 'pressure_to_sack_rate', 'avg_depth_of_target', 'avg_time_to_throw', 'ypa', 'def_gen_pressures', 'grades_hands_fumble', 'grades_offense', 'grades_pass', 'grades_run']"

In [ ]:
SEQUENCE_LENGTH = 3  # Use 3 years of history to predict the next year

# Update features based on Grouped_QB columns if needed. 
# Assuming similar structure but we will verify what's available.
# Let's stick to the previous list but be robust if columns are missing.
features = [
    'Previous_twp_rate', 
    'Previous_ypa', 
    'Previous_qb_rating', 
    'Previous_grades_pass', 
    'Value_cap_space', 
    'Previous_PFF', 
    'Previous_AV'
]
target_col = 'Current_PFF'

# Dynamic Feature checking
available_features = [f for f in features if f in df.columns]
if len(available_features) < len(features):
    print(f"Warning: Missing features. Using: {available_features}")
    features = available_features

df_clean = df.dropna(subset=features + [target_col])

# Normalize
scaler = StandardScaler()
df_clean[features] = scaler.fit_transform(df_clean[features])

def create_sequences(dataset, seq_len, features, target):
    X = []
    y = []
    
    # Group by player
    for player, group in dataset.groupby('player'):
        group = group.sort_values('Year')
        
        if len(group) <= seq_len:
            continue
            
        vals = group[features].values
        targs = group[target].values
        
        for i in range(len(group) - seq_len):
            X.append(vals[i:i+seq_len])
            y.append(targs[i+seq_len])
            
    return np.array(X), np.array(y)

# Create Datasets
X, y = create_sequences(df_clean, SEQUENCE_LENGTH, features, target_col)
print(f"Generated Sequences Shape: X={X.shape}, y={y.shape}")

if len(X) == 0:
    print("Error: Not enough history per player to create sequences of length", SEQUENCE_LENGTH)
else:
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Train: {X_train.shape}, Test: {X_test.shape}")

## 4. Build & Train Model

In [ ]:
def build_transformer_model(input_shape, head_size=32, num_heads=2, ff_dim=32, num_transformer_blocks=1, mlp_units=[64]):
    inputs = layers.Input(shape=input_shape)
    
    x = Time2Vec(kernel_size=2)(inputs)
    
    for _ in range(num_transformer_blocks):
        x = TransformerBlock(x.shape[-1], num_heads, ff_dim)(x)

    x = layers.GlobalAveragePooling1D()(x)
    
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(0.2)(x)
        
    outputs = layers.Dense(1)(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

if len(X) > 0:
    input_shape = (SEQUENCE_LENGTH, X_train.shape[2])
    model = build_transformer_model(input_shape)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
    model.summary()
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        epochs=200,
        batch_size=16,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Eval
    test_pred = model.predict(X_test).flatten()
    print(f"Test R²: {r2_score(y_test, test_pred):.4f}")
    
    plt.scatter(y_test, test_pred)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.show()